In [1]:
path_file = "../data/COLIEE2024statute_data-English/train/"

In [2]:
from os import listdir
from os.path import isfile, join

def get_all_files_from_path(mypath):
    filenames = [join(mypath, f) for f in listdir(mypath) if isfile(join(mypath, f))]
    return filenames

from bs4 import BeautifulSoup
import re
import json

def get_article(articles):
    result = {}
    current_statue = "(non-statute)"
    for i in re.split(r"(.*)", articles.strip()):
        if len(i) == 0 or i == "\n":
            continue
        if re.search(r"^\(.*\)$", i):
            current_statue = i.strip()
            if current_statue not in result:
                result.update({current_statue: []})
        else:
            if current_statue not in result:
                result.update({current_statue: []})
            result[current_statue].append(i)
    return result

def build_test(filename):
    result = {}
    with open(filename, 'r') as f:
        data = f.read()

    data = BeautifulSoup(data, "xml").find_all('pair')
    for i in data:
        id = i.get('id')
        result.update({id: {}})
        result[id].update({"label": i.get('label')})
        articles = i.find('t1').text.strip()
        # articles = get_article(articles)
        result[id].update({"result": articles})
        result[id].update({"content": i.find('t2').text.strip()})
    return result

def write_json(filename, data):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

import xml.etree.ElementTree as Et
import glob

def format_first_line(text):
    lines = text.split("\n")
    results = []
    for line in lines:
        if line[0] == "":
            continue
        if line[0] == "(" and line[-1] == ")":
            continue
        results.append(line)
    return "\n".join(results)

def load_samples(filexml):
    # try:
        tree = Et.parse(filexml)
        root = tree.getroot()
        samples = []
        for i in range(0, len(root)):
            sample = {'result': []}
            for j, e in enumerate(root[i]):
                if e.tag == "t1":
                    sample['result'] = format_first_line(e.text.strip())
                elif e.tag == "t2":
                    question = e.text.strip()
                    sample['content'] = question if len(question) > 0 else None
            sample.update(
                {'index': root[i].attrib['id'], 'label': root[i].attrib.get('label', "N")})
            # filter the noise samples
            if sample['content'] is not None:
                samples.append(sample)
            else:
                print("[Important warning] samples {} is ignored".format(sample))
        return samples

def load_test_data_samples(path_folder_base, test_id):
    data = []
    test = load_samples(f"{path_folder_base}/riteval_{test_id}.xml")
    for file_path in glob.glob(f"{path_folder_base}/riteval_{test_id}.xml"):
        data = data + load_samples(file_path)
    return data


def load_all_data_samples(path_folder_base):
    data = []
    for file_path in glob.glob("{}/*.xml".format(path_folder_base)):
        data = data + load_samples(file_path)
    return data

def check_false_labels(pred, false_labels):
	for label in false_labels:
		if label in pred:
			return True
	return False

from tqdm import tqdm

def format_output(text):
	CLEANR = re.compile('<.*?>') 
	cleantext = re.sub(CLEANR, '', text)
	return cleantext.strip().lower()

def readfile(filename):
    f = open(filename)
    data = json.load(f)
    return data

In [3]:
from transformers import AutoTokenizer, BloomForCausalLM, AutoTokenizer, AutoModelForSeq2SeqLM
import torch

model_name = "google/flan-t5-xxl"
cache_dir = "/home/congnguyen/drive/.cache"
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=cache_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(
		model_name, device_map="auto", cache_dir=cache_dir, torch_dtype=torch.float16, load_in_8bit=True
	)

/home/congnguyen/drive/miniconda3/envs/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:12<00:00,  2.41s/it]


In [26]:
from tqdm import tqdm

def format_output(text):
	CLEANR = re.compile('<.*?>') 
	cleantext = re.sub(CLEANR, '', text)
	return cleantext.strip()#.lower()

def prompting(premise, hypothesis, label, template=None):
    if "true" in template.lower():
        answer = "True"
        if "N" in label:
            answer = "False"
    else:
        answer = "Yes"
        if "N" in label:
            answer = "No"
    text = template.replace("{{premise}}", premise).replace("{{hypothesis}}", hypothesis)
    return text+"\nAnswer: "+answer+"\n\nLet't think step by step why the answer is "+answer

def write_cot(result, filename):
    data = [ujson.dumps(line, escape_forward_slashes=False) for line in result]
    with open(filename, "w", encoding="utf-8") as f:
        for line in data:
            f.write(line+'\n')

In [27]:
list_prompt = readfile("../data/prompt.json")
# list_prompt

In [28]:
import copy

def predict(model, tokenizer, files=["riteval_R01_en","riteval_R02_en","riteval_R03_en","riteval_R04_en"], output="../output/cot/newpromt_"):
    for file in files:
        test_file = path_file+file+".xml"
        data = load_samples(test_file)
        
        acc = {}
        for template_prompt in list_prompt:
            template_prompt = template_prompt["prompt"]
            print(template_prompt)
            result = []
            count = 0
            for item in tqdm(data):
                label = item["label"]
                hypothesis = item["content"]
                premise = item["result"]
                #Important: You must use dot-product, not cosine_similarity
                text = prompting(premise, hypothesis, label, template_prompt)
                inputs = tokenizer(text, return_tensors="pt")["input_ids"].cuda()
                outputs = model.generate(inputs, max_new_tokens=256)
                output_text = format_output(tokenizer.decode(outputs[0]).replace(text, "").split("\n")[-1])
                item.update({"prompt": text})
                item.update({"cot": output_text})
                result.append(item)
                if count < 5:
                    print(text)
                    print(output_text)
                    count += 1
        #     acc.update({template_prompt: count/len(data)})
        write_cot(result, output+file+"_cot.jsonl")

In [29]:
predict(model, tokenizer, ["riteval_R01_en","riteval_R02_en","riteval_R03_en","riteval_R04_en"], "../output/generated_cot/")

{{premise}}
Question: {{hypothesis}} True or False?


  1%|█▍                                                                                                                                                         | 1/111 [00:05<10:01,  5.47s/it]

Article 5
(1) A minor must obtain the consent of the minor's legal representative to perform a juridical act;provided, however, that this does not apply to a juridical act for merely acquiring a right or being released from an obligation.
(2) A juridical act in contravention of the provisions of the preceding paragraph is voidable.
(3) Notwithstanding the provisions of paragraph (1), a minor may freely dispose of property that the legal representative has permitted the minor to dispose of for a specified purpose, to an extent that falls within the scope of that purpose. The same applies if the minor disposes of property that the legal representative has permitted the minor to dispose of without specifying a purpose.
Question: A contract of sales concluded by a minor may not be rescinded if it relates to daily life, even in cases the consent of the parental authority is not obtained. True or False?
Answer: False

Let't think step by step why the answer is False
A contract of sales concl

  2%|██▊                                                                                                                                                        | 2/111 [00:11<10:41,  5.88s/it]

Article 9
A juridical act performed by an adult ward is voidable;provided, however, that this does not apply to the purchase of daily necessities or to any other act involved in day-to-day life.
Question: A contract of sales concluded by an adult ward may be rescinded except relating to daily life, even in cases the consent of the adult guardian has been obtained, the contract True or False?
Answer: True

Let't think step by step why the answer is True
Article 9 A juridical act performed by an adult ward is voidable;provided, however, that this does not apply to the purchase of daily necessities or to any other act involved in day-to-day life. So the final answer is True.


...

Article 17
(1) At the request of the person referred to in the main clause of Article 15, paragraph (1) or the assistant or assistant's supervisor, the family court may decide that the person under assistance must obtain the consent of the person's assistant in order to perform a specific juridical act;provided, however, that the acts that such a decision may establish as those for which the person must obtain the consent of the assistant are restricted to a part of the acts provided for in Article 13, paragraph (1).
(2) In order to decide as referred to in the preceding paragraph at the request of a person other than the person in question requires the consent of the person in question.
(3) If the assistant does not consent to an act for which the person under assistance must obtain the assistant's consent even though it is unlikely to prejudice the interests of the person under assistance, the family court may grant permission that operates in lieu of the assistant's consent, at the 

  5%|██████▉                                                                                                                                                    | 5/111 [01:02<18:36, 10.53s/it]

Article 124
(1) The ratification of a voidable act does not become effective unless it is made after the circumstances that made the act voidable cease to exist and the person ratifying the act becomes aware of the right to rescind it.
(2) In the following cases, the ratification referred to in the preceding paragraph is not required to be made after the circumstances that made the act voidable cease to exist:
(i) if a legal representative or a curator or assistant of a person with qualified legal capacity ratifies the act; or
(ii) if a person with qualified legal capacity (excluding an adult ward) makes the ratification with the consent of a legal representative, curator or assistant.
Article 126
The right to rescind an act is extinguished by the operation of the prescription if it is not exercised within five years from the time when it becomes possible to ratify the act. The same applies if 20 years have passed from the time of the act. Based on that information, is the claim: "The 

 70%|████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                                             | 78/111 [09:20<03:56,  7.18s/it]


KeyboardInterrupt: 

In [32]:
from os import listdir
from os.path import isfile, join

myfiles = [f.replace(".xml", "") for f in listdir(path_file) if isfile(join(path_file, f))]
# myfiles

['riteval_H18_en',
 'riteval_H30_en',
 'riteval_H29_en',
 'riteval_H25_en',
 'riteval_H22_en',
 'riteval_H21_en',
 'riteval_R04_en',
 'riteval_H24_en',
 'riteval_H28_en',
 'riteval_H20_en',
 'riteval_R01_en',
 'riteval_R02_en',
 'riteval_H26_en',
 'riteval_R03_en',
 'riteval_H23_en',
 'riteval_H27_en',
 'riteval_H19_en']